In [1]:
import datetime, warnings, scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

#import pandas
from keras.models import Sequential
from keras.layers import Dense # fully connected layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import datetime
from datetime import date
from datetime import datetime

### Loading Datasets

In [9]:
train_set = pd.read_csv('merged_flights.csv', index_col=0)
train_set.head()

,SCHEDULED_DATE,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,...,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Count Flights,Enplanements,area
0,2015-01-01,4,AS,N407AS,ANC,SEA,00:05:00,23:54:00,21.0,00:15:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
1,2015-01-01,4,AS,N309AS,ANC,SEA,00:45:00,00:41:00,17.0,00:58:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
2,2015-01-01,4,DL,N3743H,ANC,SEA,00:45:00,00:31:00,25.0,00:56:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
3,2015-01-01,4,AS,N413AS,ANC,PDX,00:50:00,00:46:00,11.0,00:57:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
4,2015-01-01,4,US,N804AW,ANC,PHX,01:52:00,01:43:00,21.0,02:04:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west


In [8]:
test_set = pd.read_csv('Data/test_set.csv', index_col=0)
test_set.head()

,SCHEDULED_DATE,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DEPARTURE_DELAY,SPEED,ARRIVAL_DELAY
id,,,,,,,,,,,,,,,,
0,2015-07-01,3,NK,N624NK,LAS,IAH,00:03:00,00:41:00,11.0,00:52:00,177.0,1222,05:00:00,38.0,414.237,0
1,2015-07-01,3,AA,N507AY,SFO,CLT,00:05:00,00:00:00,21.0,00:21:00,303.0,2296,08:08:00,-5.0,454.653,0
2,2015-07-01,3,DL,N3759,LAX,MEM,00:05:00,00:00:00,36.0,00:36:00,223.0,1619,05:48:00,-5.0,435.605,0
3,2015-07-01,3,NK,N629NK,LAS,MSP,00:05:00,00:53:00,14.0,01:07:00,178.0,1299,05:03:00,48.0,437.865,0
4,2015-07-01,3,UA,N39418,ANC,ORD,00:08:00,00:17:00,20.0,00:37:00,364.0,2846,09:12:00,9.0,469.121,0


### Dropping columns

In [10]:
train_set = train_set.drop(columns=['TAIL_NUMBER','SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','AIRPORT','COUNTRY','IATA_CODE','SCHEDULED_DATE','DAY_OF_WEEK','SCHEDULED_ARRIVAL','WHEELS_OFF','LONGITUDE','LATITUDE','Enplanements','TAXI_OUT','SPEED','Count Flights','CITY', 'STATE', 'area'])
train_set

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,ARRIVAL_DELAY,DEPARTURE_DELAY
0,AS,ANC,SEA,205.0,1448,-22.0,-11.0
1,AS,ANC,SEA,204.0,1448,-14.0,-4.0
2,DL,ANC,SEA,210.0,1448,-24.0,-14.0
3,AS,ANC,PDX,215.0,1542,-18.0,-4.0
4,US,ANC,PHX,323.0,2552,-10.0,-9.0
...,...,...,...,...,...,...,...
2818548,B6,HYA,JFK,72.0,196,-12.0,6.0
2818549,B6,HYA,JFK,72.0,196,-18.0,6.0
2818550,B6,HYA,JFK,72.0,196,-7.0,-1.0
2818551,B6,HYA,JFK,72.0,196,-22.0,-4.0


In [11]:
test_set = test_set.drop(columns=['TAIL_NUMBER','SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','SCHEDULED_DATE','DAY_OF_WEEK','SCHEDULED_ARRIVAL','WHEELS_OFF','SPEED','TAXI_OUT'])
test_set

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,ARRIVAL_DELAY
id,,,,,,,
0,NK,LAS,IAH,177.0,1222,38.0,0
1,AA,SFO,CLT,303.0,2296,-5.0,0
2,DL,LAX,MEM,223.0,1619,-5.0,0
3,NK,LAS,MSP,178.0,1299,48.0,0
4,UA,ANC,ORD,364.0,2846,9.0,0
...,...,...,...,...,...,...,...
514379,DL,PDX,DTW,248.0,1953,-3.0,0
514380,DL,PDX,ATL,274.0,2172,13.0,0
514381,F9,DEN,MCO,213.0,1546,-4.0,0


#### There are airlines, origin airports and destination airport in test_set, that are not present in train_set, so we delete them, so that the number of columns after one-hot encoding would match

In [7]:
airlines_train = train_set['AIRLINE'].unique()
airlines_test = test_set['AIRLINE'].unique()
print(f'{airlines_train}, number of airlines in train_set: {len(airlines_train)}')
print(f'{airlines_test}, number of airlines in test_set: {len(airlines_test)}')

['AS' 'DL' 'US' 'UA' 'B6' 'AA' 'NK' 'OO' 'WN' 'VX' 'HA' 'MQ' 'F9' 'EV'], number of airlines in train_set: 14
['NK' 'AA' 'DL' 'UA' 'AS' 'B6' 'HA' 'EV' 'MQ' 'OO' 'WN' 'F9' 'VX'], number of airlines in test_set: 13


In [8]:
# AIRLINE

for airline in airlines_train:
    if airline in airlines_test:
        print(f'{airline} IN')
    else:
        print(f'{airline} OUT')

AS IN
DL IN
US OUT
UA IN
B6 IN
AA IN
NK IN
OO IN
WN IN
VX IN
HA IN
MQ IN
F9 IN
EV IN


In [9]:
# Deleting rows from train_set, where AIRLINe == US
train_set = train_set.drop(train_set[(train_set['AIRLINE'] == 'US')].index)
train_set.shape

(2624330, 7)

In [ ]:
# Airports origins

In [10]:
airports_train = train_set['ORIGIN_AIRPORT'].unique()
airports_test = test_set['ORIGIN_AIRPORT'].unique()
print(len(airports_train))
print(len(airports_test))

322
312


In [11]:
out = []
for airport in airports_train:
    if airport not in airports_test:
        out.append(airport)
        
        
print(out)

['CLD', 'STC', 'BLI', 'VEL', 'PUB', 'MMH', 'CEC', 'CNY', 'ILG', 'ITH']


In [12]:
train_set = train_set.drop(train_set[(train_set['ORIGIN_AIRPORT'].isin(out))].index)
train_set.shape

(2622074, 7)

In [13]:
# Destination airports
airports_train_2 = train_set['DESTINATION_AIRPORT'].unique()
airports_test_2 = test_set['DESTINATION_AIRPORT'].unique()
print(len(airports_train_2))
print(len(airports_test_2))

322
312


In [14]:
out_2 = []
for airport in airports_train_2:
    if airport not in airports_test_2:
        out_2.append(airport)
        
print(out_2)

['CLD', 'MMH', 'CEC', 'BLI', 'PUB', 'CNY', 'VEL', 'STC', 'ILG', 'ITH']


In [15]:
train_set = train_set.drop(train_set[(train_set['DESTINATION_AIRPORT'].isin(out_2))].index)
train_set.shape

(2619823, 7)

In [16]:
#train_set.to_csv('train_set.csv')

### Load train set

In [12]:
train_set = pd.read_csv('Data/train_set.csv', index_col=0)
train_set

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,ARRIVAL_DELAY,DEPARTURE_DELAY
0,AS,ANC,SEA,205.0,1448,-22.0,-11.0
1,AS,ANC,SEA,204.0,1448,-14.0,-4.0
2,DL,ANC,SEA,210.0,1448,-24.0,-14.0
3,AS,ANC,PDX,215.0,1542,-18.0,-4.0
5,AS,ANC,SEA,218.0,1448,-35.0,-15.0
...,...,...,...,...,...,...,...
2818548,B6,HYA,JFK,72.0,196,-12.0,6.0
2818549,B6,HYA,JFK,72.0,196,-18.0,6.0
2818550,B6,HYA,JFK,72.0,196,-7.0,-1.0
2818551,B6,HYA,JFK,72.0,196,-22.0,-4.0


### One-hot encoding

In [13]:
X_train =train_set.drop(columns=['ARRIVAL_DELAY'])
Y_train = train_set['ARRIVAL_DELAY']

X_train = pd.get_dummies(X_train, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT'], prefix = ['Airline','Origin','Dest'])
X_train

,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,Airline_AA,Airline_AS,Airline_B6,Airline_DL,Airline_EV,Airline_F9,Airline_HA,...,Dest_TYR,Dest_TYS,Dest_UST,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YUM
0,205.0,1448,-11.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,204.0,1448,-4.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,210.0,1448,-14.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,215.0,1542,-4.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,218.0,1448,-15.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,72.0,196,6.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2818549,72.0,196,6.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2818550,72.0,196,-1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2818551,72.0,196,-4.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X_train.shape

(2619823, 640)

In [15]:
X_test =test_set.drop(columns=['ARRIVAL_DELAY'])
Y_test = test_set['ARRIVAL_DELAY']

X_test = pd.get_dummies(X_test, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT'], prefix = ['Airline','Origin','Dest'])
X_test

,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,Airline_AA,Airline_AS,Airline_B6,Airline_DL,Airline_EV,Airline_F9,Airline_HA,...,Dest_TYR,Dest_TYS,Dest_UST,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YUM
id,,,,,,,,,,,,,,,,,,,,,
0,177.0,1222,38.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,303.0,2296,-5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,223.0,1619,-5.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,178.0,1299,48.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,364.0,2846,9.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514379,248.0,1953,-3.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
514380,274.0,2172,13.0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
514381,213.0,1546,-4.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_test.shape

(514384, 640)

# Neural Networks

https://keras.io/api/layers/regularizers/

In [23]:
# define base model

# kernel_regularizer, bias_regularizer, activity_regularizer
# dense = tf.keras.layers.Dense(3, kernel_regularizer='l2')

def baseline_model():
    model = Sequential()
    model.add(Dense(400, input_dim=640, kernel_initializer='normal', activation='relu'))
    
    # Additional layers
    model.add(Dense(200, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))

    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam') 
    return model

In [24]:
model = baseline_model()
model.fit(X_train, Y_train, epochs=10, batch_size=50) 

Epoch 1/10
52397/52397 [==============================] - 267s 5ms/step - loss: 270.6791
Epoch 2/10
52397/52397 [==============================] - 236s 5ms/step - loss: 246.8700
Epoch 3/10
52397/52397 [==============================] - 249s 5ms/step - loss: 234.0944
Epoch 4/10
52397/52397 [==============================] - 242s 5ms/step - loss: 232.6255
Epoch 5/10
52397/52397 [==============================] - 260s 5ms/step - loss: 222.6332
Epoch 6/10
52397/52397 [==============================] - 244s 5ms/step - loss: 226.8873
Epoch 7/10
52397/52397 [==============================] - 269s 5ms/step - loss: 217.1340
Epoch 8/10
52397/52397 [==============================] - 252s 5ms/step - loss: 218.3786
Epoch 9/10
52397/52397 [==============================] - 256s 5ms/step - loss: 211.1072
Epoch 10/10
52397/52397 [==============================] - 255s 5ms/step - loss: 212.5758


### Predicting on train set

In [ ]:
# predictions = model.predict(X_train)

# print(predictions)
# print("Mean squared error - Train: %.2f" % mean_squared_error(Y_train, predictions))
# print('Variance score - Train: %.2f' % r2_score(Y_train, predictions))

### Saving model

In [1]:
# save model and architecture to single file
model.save("model_9.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

# load model
loaded_model = load_model('model_9.h5')
# summarize model.
loaded_model.summary()

In [27]:
new_predictions = loaded_model.predict(X_test)
print(new_predictions)

[[ 40.38021  ]
 [-10.870261 ]
 [ -9.2564535]
 ...
 [ -3.8454266]
 [  5.260379 ]
 [ -0.757607 ]]


#### Save file for Kaggle submition

In [28]:
pd.DataFrame(new_predictions).to_csv("submission_9.csv")
